# Lecture 12: Introduction to Keras

![](https://www.tensorflow.org/images/colab_logo_32px.png)
[Run in colab](https://colab.research.google.com/drive/1UX-aQOoLzt8q1gcGp8FQ3mM1L2qDNMA-)

In [ ]:
import datetime
now = datetime.datetime.now()
print("Last executed: " + now.strftime("%Y-%m-%d %H:%M:%S"))

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import os

2024-01-10 00:21:03.094962: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## Overview of Keras

Keras is a high-level API for deep learning.

It supports multiple back end deep learning frameworks.  

While Keras could be used with TensorFlow 1.0, as of TensorFlow 2.0 it is tightly integrated.

<br>

<img src="https://raw.githubusercontent.com/astro-informatics/course_mlbd_images/master/Lecture12_Images/keras_architecture.png" width="700px" style="display:block; margin:auto"/>

[Credit: Geron]

Keras's API is similar to Facebook's [PyTorch](https://pytorch.org/) library (PyTorch and TensorFlow are the two most popular deep learning libraries).  Both are very similar to the Scikit Learn API.  So learning Keras will be useful if you want to transition to PyTorch.

Plus, Keras considerably simplifies the use of TensorFlow!

## Sequential API

Keras supports a number of different APIs.

We'll start by considering the *Sequential API*, which is the simplest and simply considers a stack of layers connected sequentially.

### Loading data

We'll using the fashion MNIST dataset to illustrate the use of Keras.

Keras includes support to load a number of popular datasets in `keras.datasets`.

Let's use Keras to load fashion MNIST (note that the dataset already includes standard training and test sets).

In [ ]:
fashion_mnist = keras.datasets.fashion_mnist
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()

 8192/29515 [=======>......................] - ETA: 0s

29515/29515 [==============================] - 0s 0us/step


    8192/26421880 [..............................] - ETA: 0s

  466944/26421880 [..............................] - ETA: 2s

 4202496/26421880 [===>..........................] - ETA: 0s

...

26421880/26421880 [==============================] - 0s 0us/step


5148/5148 [==============================] - 0s 0us/step


   8192/4422102 [..............................] - ETA: 0s

 352256/4422102 [=>............................] - ETA: 0s

4422102/4422102 [==============================] - 0s 0us/step


The training set contains 60,000 grayscale images, each 28x28 pixels (similar to the MNIST digit dataset):

In [ ]:
X_train_full.shape

Since no validation set is provided, we'll split the full training set into a validation set and a (smaller) training set.

Let's split the full training set into a validation set and a (smaller) training set (also scale the pixel intensities from 0-255 to 0-1). 

In [ ]:
X_valid, X_train = X_train_full[:5000] / 255., X_train_full[5000:] / 255.
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]
X_test = X_test / 255.

Plot individual data instance:

In [ ]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.imshow(X_train[0], cmap="binary")
plt.axis('off')
plt.show()

The labels are the class IDs (represented as uint8), from 0 to 9:

In [ ]:
y_train

With corresponding class names:

In [ ]:
class_names = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat",
               "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]

The validation set contains 5,000 images, and the test set contains 10,000 images:

In [ ]:
X_valid.shape

In [ ]:
X_test.shape

Plot a sample of the images in the dataset:

In [ ]:
n_rows = 4
n_cols = 10
plt.figure(figsize=(n_cols * 1.2, n_rows * 1.2))
for row in range(n_rows):
    for col in range(n_cols):
        index = n_cols * row + col
        plt.subplot(n_rows, n_cols, index + 1)
        plt.imshow(X_train[index], cmap="binary", interpolation="nearest")
        plt.axis('off')
        plt.title(class_names[y_train[index]], fontsize=12)
plt.subplots_adjust(wspace=0.2, hspace=0.5)
plt.show()

### Building the model

Let's build a multi-layer perceptron (MLP) with two hidden layers:

In [ ]:
model = keras.models.Sequential()
model.add(keras.layers.Input((28, 28)))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(300, activation="relu"))
model.add(keras.layers.Dense(100, activation="relu"))
model.add(keras.layers.Dense(10, activation="softmax"))

Setting up the sequential model consists of the following steps.

1. Set up a sequential model and then we will sequentially add layers to it.
2. Flatten the 28x28 pixel image into a 1D vector.  Since it is the first layer, should specify input shape.
3. Add a fully connected dense layer with 300 neurons, including a ReLU activation function.
4. Add another fully connected dense layer with 100 neurons, including a ReLU.
5. Add a final dense layer with 10 neurons so we have one per output class, and include a softmax activation function convert to class probabilities.

Alternatively, the sequential model may be specificed by a list of layers.

In [ ]:
model = keras.models.Sequential([
    keras.layers.Input(shape=[28, 28]),
    keras.layers.Flatten(),
    keras.layers.Dense(300, activation="relu"),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(10, activation="softmax")
])

Can get a summary of the model using the `summary` method.

In [ ]:
model.summary()

_________________________________________________________________


 Layer (type)                Output Shape              Param #   


 flatten_1 (Flatten)         (None, 784)               0         


 dense_3 (Dense)             (None, 300)               235500    


 dense_4 (Dense)             (None, 100)               30100     


 dense_5 (Dense)             (None, 10)                1010      


Total params: 266610 (1.02 MB)


Trainable params: 266610 (1.02 MB)


Non-trainable params: 0 (0.00 Byte)


_________________________________________________________________


Can extract list of layers from `model.layers`.

In [ ]:
model.layers

### Compiling the model

Once the model is created we still need to specify the loss function and the optimizer to use to train the model.  This is set by *compiling* the model.

In [ ]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(learning_rate=1e-4),
              metrics=["accuracy"])

Since we have a classification problem we will use a categorical cross-entropy loss, with a stochastic gradient descent optimiser.

We will also specify an extra metric to compute during training and evlaution, in this case the accuracy.

You can find further details regarding Keras loss functions, optimisers and metrics in the Keras API.
- [Keras loss functions](https://keras.io/api/losses/)
- [Keras optimisers](https://keras.io/api/optimizers/)
- [Keras metrics](https://keras.io/api/metrics/)

### Fitting the model

Call `model.fit` to fit the model, given training and validation data and number of epochs.

The loss and extra metrics will be evaluated on the validation data at the end of each epoch.

In [ ]:
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid, y_valid))

   1/1719 [..............................] - ETA: 9:17 - loss: 2.2598 - accuracy: 0.0938

  28/1719 [..............................] - ETA: 3s - loss: 2.2947 - accuracy: 0.0960  

  58/1719 [>.............................] - ETA: 2s - loss: 2.2928 - accuracy: 0.0916

  88/1719 [>.............................] - ETA: 2s - loss: 2.2877 - accuracy: 0.0969

 115/1719 [=>............................] - ETA: 2s - loss: 2.2911 - accuracy: 0.0962

...

1615/1719 [===========================>..] - ETA: 0s - loss: 1.0058 - accuracy: 0.6978

1643/1719 [===========================>..] - ETA: 0s - loss: 1.0052 - accuracy: 0.6980

1670/1719 [============================>.] - ETA: 0s - loss: 1.0049 - accuracy: 0.6979

1696/1719 [============================>.] - ETA: 0s - loss: 1.0043 - accuracy: 0.6979

1719/1719 [==============================] - 3s 2ms/step - loss: 1.0041 - accuracy: 0.6979 - val_loss: 0.9619 - val_accuracy: 0.7080


### Model history

The fit method returns the model history, which contains the training parameters (`history.parms`) and a dictionary containing the loss and additional metrics computed during training (`history.history`).

In [ ]:
history.params

In [ ]:
history.history.keys()

Let's plot the history.

In [ ]:
import pandas as pd
pd.DataFrame(history.history).plot(figsize=(8, 5))
plt.grid(True)
#plt.gca().set_ylim(0, 1)
plt.xlabel("Epoch")
plt.show()

Note that the validation error is computed at the end of each epoch, whereas the training error is computed using a running mean during each epoch.

### Evaluating on the test set

We can evaluate the model on the test set using `model.evaluate`.

In [ ]:
model.evaluate(X_test, y_test)

 76/313 [======>.......................] - ETA: 0s - loss: 0.9815 - accuracy: 0.7015

114/313 [=========>....................] - ETA: 0s - loss: 0.9998 - accuracy: 0.6913

154/313 [=============>................] - ETA: 0s - loss: 0.9935 - accuracy: 0.6940

195/313 [=================>............] - ETA: 0s - loss: 0.9921 - accuracy: 0.6934

234/313 [=====================>........] - ETA: 0s - loss: 0.9949 - accuracy: 0.6910

273/313 [=========================>....] - ETA: 0s - loss: 0.9939 - accuracy: 0.6930

312/313 [============================>.] - ETA: 0s - loss: 0.9923 - accuracy: 0.6925

313/313 [==============================] - 0s 1ms/step - loss: 0.9921 - accuracy: 0.6924


[0.9921249747276306, 0.6923999786376953]

### Predictions

We can make predictions on new data using `model.predict`.

Consider the first three images in the test set.

In [ ]:
X_new = X_test[:3]

Estimating class probabilities:

In [ ]:
y_proba = model.predict(X_new)
y_proba.round(2)

array([[0.01, 0.01, 0.01, 0.01, 0.01, 0.21, 0.01, 0.18, 0.11, 0.45],
       [0.04, 0.01, 0.46, 0.01, 0.26, 0.  , 0.2 , 0.  , 0.02, 0.01],
       [0.01, 0.85, 0.02, 0.09, 0.01, 0.  , 0.01, 0.  , 0.  , 0.  ]],
      dtype=float32)

Making class predictions:

In [ ]:
y_pred = np.argmax(model.predict(X_new), axis=-1)
y_pred

array([9, 2, 1])

In [ ]:
np.array(class_names)[y_pred]

Check with test set targets:

In [ ]:
y_new = y_test[:3]
y_new

Let's view the images considered for prediction.

In [ ]:
plt.figure(figsize=(7.2, 2.4))
for index, image in enumerate(X_new):
    plt.subplot(1, 3, index + 1)
    plt.imshow(image, cmap="binary", interpolation="nearest")
    plt.axis('off')
    plt.title(class_names[y_test[index]], fontsize=12)
plt.subplots_adjust(wspace=0.2, hspace=0.5)
plt.show()

**Exercises:** *You can now complete Exercise 1 in the exercises associated with this lecture.*

## Functional API

While the *Sequential API* is very easy to use and many models are sequential in nature, it is also somewhat limited. 

The *Functional API* allows more complex models to be built.

For example, a Wide & Deep neural network (see [paper](https://ai.google/research/pubs/pub45413)) connects all or part of the inputs directly to the output layer.

<img src="https://raw.githubusercontent.com/astro-informatics/course_mlbd_images/master/Lecture12_Images/wide_deep_architecture.png" width="600px" style="display:block; margin:auto"/>

[Credit: Geron]

### Loading data

We will illustrate the use of the Functional API to build this model, using the California housing dataset as a regression problem.

In [ ]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()

The dataset contains 8 features (displayed below) contained in `housing.data` that can be used to predict house prices (in units of $100,000), which are contained in `housing.targets`.

In [ ]:
housing.feature_names

Split the data into training, validation and test sets.

In [ ]:
X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, random_state=42)

Scale features to standardise.

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

### Building the model

In [ ]:
input_ = keras.layers.Input(shape=X_train.shape[1:])
hidden1 = keras.layers.Dense(30, activation="relu")(input_)
hidden2 = keras.layers.Dense(30, activation="relu")(hidden1)
concat = keras.layers.concatenate([input_, hidden2])
output = keras.layers.Dense(1)(concat)
model = keras.models.Model(inputs=[input_], outputs=[output])

Setting up the functional model consists of the following steps.

1. Set up the input of the model.
2. Create a dense layer with 30 neurons that takes `_input` as the input, and includes a ReLU activation function.
3. Create a dense layer with 30 neurons that takes the output of the previous layer as an input, and includes a ReLU activation function.
4. Concatenate the input and the output of the second hidden layer.
5. Create a dense layer that takes the concatenated data as an input and outputs a single value (for the regression model).
6. Build the model specifying the inputs and outputs (of which there can be multiple).

Note that each component is called like a function, mapping inputs to outputs, hence the name of the *Functional API*.

In [ ]:
model.summary()

__________________________________________________________________________________________________


 Layer (type)                Output Shape                 Param #   Connected to                  


 input_1 (InputLayer)        [(None, 8)]                  0         []                            


 dense_6 (Dense)             (None, 30)                   270       ['input_1[0][0]']             


 dense_7 (Dense)             (None, 30)                   930       ['dense_6[0][0]']             


...

 dense_8 (Dense)             (None, 1)                    39        ['concatenate[0][0]']         


Total params: 1239 (4.84 KB)


Trainable params: 1239 (4.84 KB)


Non-trainable params: 0 (0.00 Byte)


__________________________________________________________________________________________________


### Compiling the model

Once the model is created we again need to *compile* the mode to specify the loss function and the optimizer to use to train the model.  

In [ ]:
model.compile(loss="mean_squared_error", optimizer=keras.optimizers.SGD(learning_rate=1e-4))

### Fitting the model

Again we fit the model, given training and validation data and number of epochs, and compute the loss on the validation data at the end of each epoch.

In [ ]:
history = model.fit(X_train, y_train, epochs=20,
                    validation_data=(X_valid, y_valid))

  1/363 [..............................] - ETA: 1:27 - loss: 5.0352

 59/363 [===>..........................] - ETA: 0s - loss: 4.4361  

116/363 [========>.....................] - ETA: 0s - loss: 4.4623

175/363 [=============>................] - ETA: 0s - loss: 4.2806

227/363 [=================>............] - ETA: 0s - loss: 4.1482

...

179/363 [=============>................] - ETA: 0s - loss: 0.6647

237/363 [==================>...........] - ETA: 0s - loss: 0.6587

296/363 [=======================>......] - ETA: 0s - loss: 0.6589

354/363 [============================>.] - ETA: 0s - loss: 0.6605

363/363 [==============================] - 0s 1ms/step - loss: 0.6606 - val_loss: 0.6268


### Evaluating on the test set

We evaluate the model on the test set as before.

In [ ]:
mse_test = model.evaluate(X_test, y_test)

 64/162 [==========>...................] - ETA: 0s - loss: 0.6827

128/162 [======================>.......] - ETA: 0s - loss: 0.6675

162/162 [==============================] - 0s 815us/step - loss: 0.6546


### Predictions

We again make predictions on new data (the first three instances of the test set) using `model.predict`.

In [ ]:
X_new = X_test[:3]
y_pred = model.predict(X_new)
y_pred

array([[0.8628876],
       [1.0634453],
       [2.492951 ]], dtype=float32)

## Subclassing API

Both the *Sequential API* and the *Functional API* support static models: they involving defining a model and then feeding it data for training or inference.

The *Subclassing API* allows you to subclass Keras models (and other objects such as layers).  This supports dynamic models and allows you to construct alternative model objects that can be widely used elsewhere.

### Building the model

Let's use the *Subclassing API* to build a Wide & Deep neural network with two inputs and two outputs as an example.

<img src="https://raw.githubusercontent.com/astro-informatics/course_mlbd_images/master/Lecture12_Images/wide_deep_architecture_multiple_io.png" width="600px" style="display:block; margin:auto"/>

[Credit: Geron]

Firstly, let's spit our data so we have two sets of inputs.

In [ ]:
X_train_A, X_train_B = X_train[:, :5], X_train[:, 2:]
X_valid_A, X_valid_B = X_valid[:, :5], X_valid[:, 2:]
X_test_A, X_test_B = X_test[:, :5], X_test[:, 2:]
X_new_A, X_new_B = X_test_A[:3], X_test_B[:3]

A model class can then be written to implement the model, inheriting from `keras.models.Model`.

In [ ]:
class WideAndDeepModel(keras.models.Model):
    def __init__(self, units=30, activation="relu", **kwargs):
        super().__init__(**kwargs)
        self.hidden1 = keras.layers.Dense(units, activation=activation)
        self.hidden2 = keras.layers.Dense(units, activation=activation)
        self.main_output = keras.layers.Dense(1)
        self.aux_output = keras.layers.Dense(1)
        
    def call(self, inputs):
        input_A, input_B = inputs
        hidden1 = self.hidden1(input_B)
        hidden2 = self.hidden2(hidden1)
        concat = keras.layers.concatenate([input_A, hidden2])
        main_output = self.main_output(concat)
        aux_output = self.aux_output(hidden2)
        return main_output, aux_output

model = WideAndDeepModel(30, activation="relu")

The model must have `__init__` and `call` methods.  Within `__init__` you need to instantiate the layers that you need, before you make use of them within `call`.

Since the `call` method is implemented directly, there is a great deal of flexibility to implement complex models.

The model can then be used like other Keras model and you can compile and fit it as usual.

Note that for bespoke models implemented this way, Keras cannot easily inspect it.  Thus, it is not so straightforward, e.g., to save the model or to perform type and shape checking.  

### Compiling the model

In [ ]:
model.compile(loss=["mse", "mse"], loss_weights=[0.9, 0.1], optimizer=keras.optimizers.SGD(learning_rate=1e-4))

### Fitting the model

In [ ]:
history = model.fit((X_train_A, X_train_B), (y_train, y_train), epochs=20,
                    validation_data=((X_valid_A, X_valid_B), (y_valid, y_valid)))

  1/363 [..............................] - ETA: 2:53 - loss: 5.9495 - output_1_loss: 6.0951 - output_2_loss: 4.6391

 52/363 [===>..........................] - ETA: 0s - loss: 6.0571 - output_1_loss: 6.1273 - output_2_loss: 5.4251  

102/363 [=======>......................] - ETA: 0s - loss: 5.8515 - output_1_loss: 5.9350 - output_2_loss: 5.0999

151/363 [===========>..................] - ETA: 0s - loss: 5.6583 - output_1_loss: 5.7532 - output_2_loss: 4.8042

199/363 [===============>..............] - ETA: 0s - loss: 5.5574 - output_1_loss: 5.6443 - output_2_loss: 4.7752

...

200/363 [===============>..............] - ETA: 0s - loss: 0.9138 - output_1_loss: 0.8049 - output_2_loss: 1.8944

252/363 [===================>..........] - ETA: 0s - loss: 0.9145 - output_1_loss: 0.8067 - output_2_loss: 1.8843

302/363 [=======================>......] - ETA: 0s - loss: 0.9328 - output_1_loss: 0.8239 - output_2_loss: 1.9132

353/363 [============================>.] - ETA: 0s - loss: 0.9433 - output_1_loss: 0.8266 - output_2_loss: 1.9939

363/363 [==============================] - 0s 1ms/step - loss: 0.9424 - output_1_loss: 0.8257 - output_2_loss: 1.9931 - val_loss: 1.9300 - val_output_1_loss: 1.7766 - val_output_2_loss: 3.3108


### Evaluating and making predictions

In [ ]:
total_loss, main_loss, aux_loss = model.evaluate((X_test_A, X_test_B), (y_test, y_test))
y_pred_main, y_pred_aux = model.predict((X_new_A, X_new_B))

 52/162 [========>.....................] - ETA: 0s - loss: 0.9102 - output_1_loss: 0.8088 - output_2_loss: 1.8225

107/162 [==================>...........] - ETA: 0s - loss: 0.8964 - output_1_loss: 0.7964 - output_2_loss: 1.7962

162/162 [==============================] - 0s 934us/step - loss: 0.9276 - output_1_loss: 0.7995 - output_2_loss: 2.0800


1/1 [==============================] - ETA: 0s

1/1 [==============================] - 0s 53ms/step


## Saving and restoring models

Keras can be used to easily save and restore models when using the *Sequential API* of the *Functional API*.

Models can of course be saved and restored when using the *Subclassing API* but it is necessary to use underlying TensorFlow methods and is not quite so straightforward (not covered further here).

Let's build and train a simple model and then save and restore it.

In [ ]:
model = keras.models.Sequential([
    keras.layers.Input(shape=[8]),
    keras.layers.Dense(30, activation="relu"),
    keras.layers.Dense(30, activation="relu"),
    keras.layers.Dense(1)
])    

In [ ]:
model.compile(loss="mse", optimizer=keras.optimizers.SGD(learning_rate=1e-4))
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))
mse_test = model.evaluate(X_test, y_test)

  1/363 [..............................] - ETA: 1:23 - loss: 4.5745

 62/363 [====>.........................] - ETA: 0s - loss: 4.7668  

119/363 [========>.....................] - ETA: 0s - loss: 4.6584

175/363 [=============>................] - ETA: 0s - loss: 4.4761

235/363 [==================>...........] - ETA: 0s - loss: 4.3275

...

363/363 [==============================] - 0s 1ms/step - loss: 0.7680 - val_loss: 1.3381


  1/162 [..............................] - ETA: 2s - loss: 0.8639

 61/162 [==========>...................] - ETA: 0s - loss: 0.7393

125/162 [======================>.......] - ETA: 0s - loss: 0.7472

162/162 [==============================] - 0s 828us/step - loss: 0.7463


### Saving

Saving the model is simply one line.

In [ ]:
model.save("my_keras_model.keras")

The model is then saved using the Keras format.

The model's achitecture, values of the model parameters, the optimiser, hyperparameters, and state are all saved.

### Restoring

Restoring the model is also just one line.

In [ ]:
model = keras.models.load_model("my_keras_model.keras")

The model can then be used as usual, e.g. to make predictions.

In [ ]:
model.predict(X_new)

array([[1.0386337],
       [1.1102493],
       [2.4406295]], dtype=float32)

### Checkpointing

Since training may take a long time, you might like to save the model weights as training progresses so that you do not lose the results if the machine crashes or disconnects.

Model weights can also be saved and loaded in a single line.

In [ ]:
model.save_weights("my_keras.weights.h5")

In [ ]:
model.load_weights("my_keras.weights.h5")

## TensorBoard

TensorBoard is an interactive visualisation tool that you can use to review learning curves, analyse training statistics and inspect the architecture of your model.

### Logging

Let's log a training run and model and then view in TensorBoard.

In [ ]:
root_logdir = os.path.join(os.curdir, "my_logs")

In [ ]:
def get_run_logdir():
    import time
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    return os.path.join(root_logdir, run_id)

run_logdir = get_run_logdir()
run_logdir

In [ ]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
model = keras.models.Sequential([
    keras.layers.Input(shape=[8]),
    keras.layers.Dense(30, activation="relu"),
    keras.layers.Dense(30, activation="relu"),
    keras.layers.Dense(1)
])    
model.compile(loss="mse", optimizer=keras.optimizers.SGD(learning_rate=1e-4))

To log training as it progresses we need to set up `callback`.  

The `fit` methods allows a list of `callbacks` to be specified, which can then be called at the start and end of each epoch.

Let's set up two callbacks:
- One to checkpoint the model as we go, saving the best model considered so far, as evaluated on the validation set.
- One to log TensorBoard data.

In [ ]:
checkpoint_cb = keras.callbacks.ModelCheckpoint("my_keras_model.keras", save_best_only=True)
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)

The callbacks are then simply passed to the `fit` method.

In [ ]:
history = model.fit(X_train, y_train, epochs=30,
                    validation_data=(X_valid, y_valid),
                    callbacks=[checkpoint_cb, tensorboard_cb])

  1/363 [..............................] - ETA: 1:27 - loss: 7.0073

 53/363 [===>..........................] - ETA: 0s - loss: 6.0794  

104/363 [=======>......................] - ETA: 0s - loss: 5.9392

154/363 [===========>..................] - ETA: 0s - loss: 5.7578

203/363 [===============>..............] - ETA: 0s - loss: 5.5502

...

204/363 [===============>..............] - ETA: 0s - loss: 0.6809

254/363 [===================>..........] - ETA: 0s - loss: 0.6710

304/363 [========================>.....] - ETA: 0s - loss: 0.6687

356/363 [============================>.] - ETA: 0s - loss: 0.6747

363/363 [==============================] - 1s 1ms/step - loss: 0.6732 - val_loss: 0.7113


### Visualisation

TensorBoard then needs to be run to visualise the logs.  It simply runs in a browser.

TensorBoard can be launched from your terminal by:

```bash
tensorboard --logdir=./my_logs --port=6001
```

We can then launch TensorBoard in a browser at [localhost:6001](http://localhost:6001).

Let's switch to TensorBoard to view the logs and inspect the computational graph of the model.